In [34]:
import pandas as pd
import glob
import numpy as np

In [37]:
class clean_and_combine:
    def __init__(self, threshold=5, max_missing=384370):
        self.threshold = threshold
        self.max_missing = max_missing

    def clean_NMR(self, df_NMR):
        # removing first column
        df_NMR = df_NMR.iloc[:, 1:]

        # remove eid so it doesn't get transformed
        eid_col = df_NMR.iloc[:,0]
        df_NMR = df_NMR.iloc[:,1:]

        words_to_remove = ['Visit2', 'Visit3', 'Visit4']
        columns_to_keep = ~df_NMR.columns.str.contains('|'.join(words_to_remove))
        df_NMR = df_NMR.loc[:, columns_to_keep]

        df_NMR = df_NMR.drop(columns=['Date_of_attending_assessment_centre_Visit1_0'])

        print(df_NMR.shape)
        print("Missing values per column before dropping:", df_NMR.isna().sum())
        df_cols = df_NMR.isna().sum()
        df_cols.to_csv('data/cleaning data/NMR_missing_values.csv', index=False)
        df_NMR = df_NMR.loc[:, df_NMR.isna().sum() <= self.max_missing]
        print(df_NMR.shape)

        # mean imputation
        df_NMR = df_NMR.fillna(df_NMR.mean())

        # remove val > 5std
        df_NMR_filt = df_NMR.copy()
        for col in df_NMR.columns:
            mean = df_NMR[col].mean()
            std = df_NMR[col].std()
            df_NMR_filt.loc[abs(df_NMR[col] - mean) > self.threshold * std, col] = np.nan
        print(df_NMR_filt.shape)

        # add 1
        df_NMR_filt = df_NMR_filt.add(1, axis='index')

        # log
        df_NMR_filt = df_NMR_filt.apply(lambda x: np.log(x))

        # normalize
        df_NMR_filt = df_NMR_filt.apply(lambda x: (x - x.mean()) / x.std())

        # remove _Visit1_0 from cols
        df_NMR_filt.columns = df_NMR_filt.columns.str.replace('_Visit1_0', '')

        df_NMR_filt = pd.concat([eid_col, df_NMR_filt], axis=1)

        df_NMR_filt.to_csv('data/cleaning data/NMR_cleaned.csv', index=False)
        return df_NMR_filt
    
    def merge_BIG(self, pattern):
        csv_files = glob.glob(pattern)
        df_BIG_merged = None # initialise data
        for csv_file in csv_files:
            df = pd.read_csv(csv_file)
            if df_BIG_merged is None:
                df_BIG_merged = df  # first dataframe initializes df_BIG_merged
            else:
                df_BIG_merged = pd.concat([df_BIG_merged, df], ignore_index=True)
        
        df_BIG_merged.to_csv('data/cleaning data/BIG_merged.csv', index=False)
        
        return df_BIG_merged
    
    def clean_BIG(self, df_BIG_merged):
        eid_col = df_BIG_merged.iloc[:, 0]
        df_BIG_merged = df_BIG_merged.iloc[:, 1:]
        
        headers_to_keep = ['X34.0.0', 'Gender', 'Ethnicity', 'Migrant.Status', 'TDI.Tertiles', 'Highest.Qualification', 'House.Ownership', 'Income', 'Cohabiting', 'Living.Alone', 'AUDIT.Score', 'Smoker', 'Moderate.Physical.Activity', 'Longstanding.Illness', 'Diabetes', 'Cancer',
                            'CVD','Depression variables','Depressed.At.Baseline','Loneliness','Social.Isolation','PHQ9.Screen','PHQ9.No.Info','PHQ9.Items','PHQ9.Severity','CIDI.MDD.No.Info','CIDI.MDD.Screen','CIDI.MDD.Response','CIDI.MDD.Severity','GAD.CIDI.Somatic']
        # Covariets, MDD, and CVD variables

        df_BIG_cleaned = df_BIG_merged[[col for col in df_BIG_merged.columns if col in headers_to_keep]].copy()
        df_BIG_cleaned.replace('?', np.nan, inplace=True)
        threshold = len(df_BIG_cleaned) * 0.5
        df_BIG_cleaned = df_BIG_cleaned.dropna(axis=1, thresh=threshold)

        df_BIG_cleaned = pd.concat([eid_col, df_BIG_cleaned], axis=1)
        
        df_BIG_cleaned.to_csv('data/cleaning data/BIG_cleaned.csv', index=False)
        return df_BIG_cleaned
    
    def combine_NMR_BIG(self, df_NMR_cleaned, df_BIG_cleaned):
        df_combined = pd.merge(df_NMR_cleaned, df_BIG_cleaned[df_BIG_cleaned['eid'].isin(df_NMR_cleaned['eid'])], on='eid', how='inner')
        df_combined.to_csv('data/cleaning data/participant_combined.csv', index=False)
        return df_combined

    

Run Classes

In [38]:
def main():
    # Create an instance of clean_and_combine
    cleaner = clean_and_combine()

    # Read and clean NMR data
    df_NMR = pd.read_csv('data/initial data/NMR_metabolic_biomarkers_with_dates.csv', delimiter=',')
    df_NMR_cleaned = cleaner.clean_NMR(df_NMR)
    print("NMR Data Cleaned: ", df_NMR_cleaned.head())

    # Merge BIG data
    pattern = 'data/big data/full*.csv'
    df_BIG_merged = cleaner.merge_BIG(pattern)
    print("BIG Data Merged: ", df_BIG_merged.shape)

    # Clean BIG data
    df_BIG_cleaned = cleaner.clean_BIG(df_BIG_merged)
    print("BIG Data Cleaned: ", df_BIG_cleaned.shape)

    # Combine NMR and BIG data
    df_combined = cleaner.combine_NMR_BIG(df_NMR_cleaned, df_BIG_cleaned)
    print("Combined Data: ", df_combined.shape)

if __name__ == "__main__":
    main()



(502379, 168)
Missing values per column before dropping: Total_Cholesterol_Visit1_0                                      384370
Total_Cholesterol_Minus_HDL-C_Visit1_0                          384370
Remnant_Cholesterol_(Non-HDL,_Non-LDL_-Cholesterol)_Visit1_0    384370
VLDL_Cholesterol_Visit1_0                                       384370
Clinical_LDL_Cholesterol_Visit1_0                               384370
                                                                 ...  
Phospholipids_in_Small_HDL_Visit1_0                             384370
Cholesterol_in_Small_HDL_Visit1_0                               384370
Cholesteryl_Esters_in_Small_HDL_Visit1_0                        384370
Free_Cholesterol_in_Small_HDL_Visit1_0                          384370
Triglycerides_in_Small_HDL_Visit1_0                             384370
Length: 168, dtype: int64
(502379, 135)
(502379, 135)
NMR Data Cleaned:         eid  Total_Cholesterol  Total_Cholesterol_Minus_HDL-C  \
0  1000013           0.05

C:\Users\elizl\AppData\Local\Temp\ipykernel_20228\3581798093.py:59: DtypeWarning: Columns (3,4,10,12,14,22,26,31,34,35,36,37,38,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,70,72,73,74,76,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)
C:\Users\elizl\AppData\Local\Temp\ipykernel_20228\3581798093.py:59: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)
C:\Users\elizl\AppData\Local\Temp\ipykernel_20228\3581798093.py:59: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)
C:\Users\elizl\AppData\Local\Temp\ipykernel_20228\3581798093.py:59: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)
C:\Users\elizl\AppData\Local\Temp\ipykernel_20228\3581798093.py:59: DtypeWarning: Colum

BIG Data Merged:  (502379, 293)
BIG Data Cleaned:  (502379, 30)
Combined Data:  (502379, 165)
